#### Import libraries and Enable GPU

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
from matplotlib import pyplot
import MNISTtools
import torch.utils.data as data_utils
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

#### Import features and labels to numpy arrays. 
#### Normalize images between [-1,1] 
#### Convert labels to one-hot encoding 

In [ ]:
xtrain, ltrain = MNISTtools.load(dataset='training', path = '/datasets/MNIST')
xtrain = np.transpose(xtrain)

def normalize_MNIST_images(x):
    x = x.astype(np.float32)
    x = (x - 127.5)/127.5
    return x

xtrain = normalize_MNIST_images(xtrain).reshape(60000,1,28,28)


def label2onehot(lbl):
    d = np.zeros((lbl.max() + 1, lbl.size))
    d[lbl[np.arange(0, lbl.size)], np.arange(0, lbl.size)] = 1
    return d
dtrain = label2onehot(ltrain)

dtrain = np.transpose(dtrain)
print(xtrain.shape)
print(dtrain.shape)

#### Load numpy arrays as tensors onto the GPU

In [ ]:
img = torch.from_numpy(xtrain).float().to(device)
lbl = torch.from_numpy(dtrain).float().to(device)
'''Below is for mini batch gradient decent '''
train = data_utils.TensorDataset(img,lbl)
data_loader = data_utils.DataLoader(train, batch_size = 100, shuffle = True)
print(img.size())
print(lbl.size())
print(type(img))
print(type(lbl))

#### Set the hyper parameters and define cross entropy loss

In [ ]:
#Hyperparameters
num_classes = 10
num_epochs = 2
batch_size = 500
learning_rate = 0.001

def cross_entropy(input, target):
    logsoftmax = nn.LogSoftmax(dim = 1)
    return torch.mean(torch.mean(-target * logsoftmax(input), dim=1))

#### Initialize the model 

In [ ]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(nn.Linear(5*5*32, num_classes),nn.Softmax() )
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.layer3(out)
        return out

model = ConvNet(num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

#### Define miss classifications 

In [ ]:
def miss_classifications(yinit, lbl):
    _, predicted = torch.max(yinit.data, 1)
    miss_classifications = (predicted != torch.max(lbl.data, 1)[1]).sum().item()
    return miss_classifications

#### Model Training 

In [ ]:
for epoch in range(num_epochs):
    mc = 0
    for i, (images, labels) in enumerate(data_loader):
        outputs = model(images)
        loss = cross_entropy(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        mc += miss_classifications(outputs, labels)
        print ('Epoch: {}, Loss: {}'.format(epoch+1, loss.item()))
    print("Miss Classifications:", (mc/600))
torch.cuda.empty_cache()